In [2]:
import numpy as np
import tensorflow as tf

import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.colors as colors 
%matplotlib notebook
%matplotlib inline

import ipysh

%aimport models.base

import Hunch_utils  as Htls
import Hunch_lsplot as Hplt
import Hunch_tSNEplot as Hsne

%aimport Dataset_QSH

%aimport models.AEFIT5
%aimport models.Compose

# ipysh.Bootstrap_support.debug()

# TEST QSH


In [3]:
qsh = Dataset_QSH.Dataset_QSH()
import os
file = ipysh.abs_builddir+'/te_db_r15_clean.npy'
try: qsh.load(file)
except: raise FileNotFoundError
    
qsh.shuffle()
qsh.dim = None
qsh.set_null(np.nan)
qsh.set_normal_positive(['prel','te','tbordo','tcentro', 'Ip','NS','VT','F'])
# qsh.set_normal_positive(['prel','te','tbordo','tcentro', 'Ip','NS','VT','F','absBr_rm','argBr_rm'])
# qsh.unbias_mean(0.5, 'te')
# qsh.set_normal_positive(['te'])
# qsh.clip_values(0.1,0.6)
# qsh.set_normal_positive(['te'])


print("QSH rebalanced 15 points size: ", len(qsh))

QSH rebalanced 15 points size:  47567


In [4]:
Br_min, Br_max = np.nanmin(qsh['Br_rm']), np.nanmax(qsh['Br_rm'])
def _map(xy,p,Br):
    Br = (Br-Br_min)/(Br_max-Br_min)
    pBr = tf.concat([p,Br], axis=0)
    return (xy,pBr),(xy,pBr)
# ds = qsh.tf_tuple_compose(['prel','te','tbordo~tcentro~Ip~NS~VT~F','absBt_rm~argBt_rm']).map(lambda x,y,t,s: (_map(x,y,t,s)) )
# ds = qsh.tf_tuple_compose(['prel~te:15','Ip~NS~VT~F','absBr_rm~argBr_rm']).map(lambda x,y,z: _map(x,y,z) )
ds = qsh.tf_tuple_compose(['prel~te:15','Ip~NS~VT~F','Br_rm']).map(lambda x,y,z: _map(x,y,z) )
# [x for x in ds.shuffle(100).batch(1).take(1)][0]

W0830 09:48:35.417568 140034843486016 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/data/ops/dataset_ops.py:504: py_func (from tensorflow.python.ops.script_ops) is deprecated and will be removed in a future version.
Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, there are two
    options available in V2.
    - tf.py_function takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well as
    being differentiable using a gradient tape.
    - tf.numpy_function maintains the semantics of the deprecated tf.py_func
    (it is not differentiable, and manipulates numpy arrays). It drops the
    stateful argument making all functions stateful.
    


In [6]:
m1 = models.AEFIT5.AEFIT5(latent_dim=10, feature_dim=30,  dprate=0.5, scale=1, geometry=[20,20,10], beta=0.) #beta=0.001)
m2 = models.AEFIT5.AEFIT5(latent_dim=24, feature_dim=24,  dprate=0.2, scale=1, beta=0., name='parameters', geometry=[1]) # parameters
hm_feature_dim = m1.latent_dim + m2.latent_dim
hm = models.AEFIT5.AEFIT5(latent_dim=10, feature_dim=hm_feature_dim, beta=0., scale=1, name='hidden', geometry=[20,80,80,50])
h = models.Compose.Compose().set_model(hm).compose([m1,m2])

AEFIT5 a ready:
AEFIT5 a ready:
AEFIT5 a ready:
[(None, 30), (None, 24)]


In [7]:
ds_m1 = qsh.tf_tuple_compose(['prel~te:15']).map(lambda x: (x,x) )
models.base.train_thread(m1, ds_m1, batch=100, epoch=10, callbacks=[]).control_panel()

In [8]:
# m1.save('step12_m1_compose')
m1.load('step12_m1_compose')

In [9]:
m1.trainable = False
m2.trainable = True
hm.trainable = True
h.loss_weights = [0.6,0.4]

# l1 = tf.keras.regularizers.l1
# l2 = tf.keras.regularizers.l2
# for n in [hm.inference_net, hm.generative_net]:
#     for l in n.layers:
#         if issubclass(type(l), tf.keras.layers.Dense):
#             l.activity_regularizer = l1(0.01)
            
h.compile( )

In [12]:
import livelossplot.keras
class PlotLossesCallback(livelossplot.keras.PlotLossesCallback):
    def on_train_batch_begin(self, a, b): pass
    def on_train_batch_end(self, a, b): pass

# hm.beta = 1.
# h.compile()
models.base.train_thread(h, ds.take(10000) , epoch=100, batch=100, learning_rate=1e-3, callbacks=[PlotLossesCallback()]).control_panel()
# h.fit(ds.batch(100), epochs=6)

/home/andrea/devel/rfx/rfx-hunch/build/conf/python/site-packages/lib/python3.6/site-packages/livelossplot/core.py:15: UserWarning: livelossplot requires inline plots.
Your current backend is: nbAgg
Run in a Jupyter environment and execute '%matplotlib inline'.
  warnings.warn("livelossplot requires inline plots.\nYour current backend is: {}\nRun in a Jupyter environment and execute '%matplotlib inline'.".format(backend))


Epoch 1/100
     75/Unknown - 18s 244ms/step - loss: 30.8500 - output_1_loss: 15.7615 - output_2_loss: 15.0886

Exception in thread async train:
Traceback (most recent call last):
  File "/usr/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/home/andrea/devel/rfx/rfx-hunch/build/conf/python/site-packages/lib/python3.6/site-packages/nbmultitask.py", line 148, in run
    fn(*self._args,**self._kwargs)
  File "/home/andrea/devel/rfx/rfx-hunch/src/Tprofile_read/models/base.py", line 249, in <lambda>
    fn = lambda thread_print: train(model, data, **kwargs)
  File "/home/andrea/devel/rfx/rfx-hunch/src/Tprofile_read/models/base.py", line 239, in train
    history = model.fit(data, epochs=epoch, callbacks=callbacks + tensorboard_log(log_name), verbose=1)
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py", line 643, in fit
    use_multiprocessing=use_multiprocessing)
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training_generator.py", line 694, in fit
    steps_name='steps_per_epoch')
  File "/usr

In [ ]:
# [ v.name for v in h2.trainable_variables ]
def plot(xy):
    plt.figure('data')
    ax = plt.gca()
    ax.set_ylim(0.,.8)
    x,y = tf.split(xy[0], num_or_size_splits=2)
    plt.plot(x,y,'.')
d = [x for x in ds.shuffle(100).batch(1).take(100)][0]
d0 = d[0][0]
d_param = ( tf.zeros_like(d[0][0]), d[0][1] ) # spengo SXR e tengo i parametri
# y0 = h(d[0], training=False)
y0 = h(d_param, training=False)
y0 = tf.sigmoid(y0[0])
plot(d0)
plot(y0)



In [ ]:
# tbordo tcentro Ip NS VT F
xy,_ =  [x for x in ds.batch(1).take(1)][0]
par = xy[2]

l,_ = h.encode(xy, training=False)
XY  = h.decode(l, training=False, apply_sigmoid=True) 
PAR = XY[2]

print( list(zip(par, PAR)) )


In [ ]:
fig = plt.figure('test_curve',figsize=(18, 6))
plt.clf()
ax1 = fig.add_subplot(121)
ax2 = fig.add_subplot(122)    
# ax1.set_xlim(-2.,2.)
ax2.set_ylim(0.,1.)

# sx = []
# sy = []
# for xy in ds.batch(1).take(1000):
#     xy,_ = xy
#     x,y = tf.split(xy[0],2, axis=1)
#     me,_  = h.encode(xy, training=False)
#     gpt = me[0].numpy()
#     #ax1.scatter(gpt[0],gpt[1])
#     sx.append(gpt[0])
#     sy.append(gpt[1])

# ax1.scatter(sx,sy)
    
for xy in ds.shuffle(100).batch(1).take(1):    
    xy,_ = xy
    x,y = tf.split(xy[0],2, axis=1)
    ax2.scatter(x,y,s=80)
    me,_  = h.encode(xy, training=False)
    gpt = me[0].numpy()
    ax1.scatter(gpt[0],gpt[1])
    
    XY = h.decode(me, training=False)[0]
    XY = tf.sigmoid(XY)
    X,Y = tf.split(XY[0], num_or_size_splits=2)
    X,Y = (X.numpy(), Y.numpy())
    ax2.scatter(X,Y,s=40)

print(qsh_pos)    
qsh_pos += 1

In [ ]:
xy,_ = [x for x in ds.batch(2).take(1)][0]
me,va = h.encode(xy)
XY = h.decode(me, apply_sigmoid=True)
XY[0][0]